In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    TimeDistributed, BatchNormalization, LSTM
)
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
from sklearn.model_selection import train_test_split

import optuna


### CNN Auto Hyperparameter Tuning

In [ ]:

# Load data dari file .npz
data_train_cnn = np.load('CNN_train_data.npz')

X = data_train_cnn['data']
y = data_train_cnn['label']

# Ambil 50% data secara acak tetapi tetap representatif
X_half, _, y_half, _ = train_test_split(X, y, test_size=0.5, random_state=42)

# Bagi lagi menjadi training dan validation set (80% : 20%)
X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
    X_half, y_half, test_size=0.2, random_state=42
)

# Verifikasi ukuran
print(f"Total sample used: {len(X_half)}")
print(f"Train size: {len(X_train_sub)}, Validation size: {len(X_val_sub)}")


Total sample used: 11234
Train size: 8987, Validation size: 2247


In [ ]:
# Cek distribusi kelas pada seluruh subset (50%)
print("Distribusi kelas pada subset 50% data:")
classes, counts = np.unique(y_half, return_counts=True)
for c, n in zip(classes, counts):
    print(f"  Kelas {int(c)}: {n} sampel")

# Cek distribusi kelas pada data training
print("\nDistribusi kelas pada data training:")
classes_train, counts_train = np.unique(y_train_sub, return_counts=True)
for c, n in zip(classes_train, counts_train):
    print(f"  Kelas {int(c)}: {n} sampel")

# Cek distribusi kelas pada data validasi
print("\nDistribusi kelas pada data validasi:")
classes_val, counts_val = np.unique(y_val_sub, return_counts=True)
for c, n in zip(classes_val, counts_val):
    print(f"  Kelas {int(c)}: {n} sampel")


Distribusi kelas pada subset 50% data:
  Kelas 0: 5014 sampel
  Kelas 1: 6220 sampel

Distribusi kelas pada data training:
  Kelas 0: 4010 sampel
  Kelas 1: 4977 sampel

Distribusi kelas pada data validasi:
  Kelas 0: 1004 sampel
  Kelas 1: 1243 sampel


In [ ]:
# === Objective Function ===
def objective(trial):

    # Hyperparameters to tune
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    dropout_rate = trial.suggest_categorical("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Model architecture
    model = Sequential([
        Conv2D(8, (3, 3), activation='relu', input_shape=(128, 128, 1), name='conv2d_1'),
        MaxPooling2D(2, 2, name='maxpool_1'),
        Conv2D(16, (3, 3), activation='relu', name='conv2d_2'),
        MaxPooling2D(2, 2, name='maxpool_2'),
        Flatten(name='flatten'),
        Dense(32, activation='relu', name='dense_1'),
        Dropout(dropout_rate, name='dropout_1'),
        Dense(1, activation='sigmoid', name='output')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # Fit model
    history = model.fit(
        X_train_sub, y_train_sub,
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_val_sub, y_val_sub),
        callbacks=[early_stop],
        verbose=1
    )

    # Return validation accuracy
    val_acc = max(history.history["val_accuracy"])
    return val_acc



In [ ]:
# === Run Optuna Study ===
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30)

# === Output best result ===
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-05-12 14:26:31,934] A new study created in memory with name: no-name-09762fbf-3769-4876-a6d9-050c0d0f1189


Epoch 1/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7306 - loss: 0.5138 - val_accuracy: 0.7859 - val_loss: 0.4170
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7921 - loss: 0.4057 - val_accuracy: 0.8202 - val_loss: 0.3787
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8261 - loss: 0.3819 - val_accuracy: 0.8603 - val_loss: 0.3404
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8570 - loss: 0.3283 - val_accuracy: 0.8554 - val_loss: 0.3206
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8591 - loss: 0.3152 - val_accuracy: 0.8856 - val_loss: 0.2977
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8851 - loss: 0.2768 - val_accuracy: 0.8936 - val_loss: 0.2633
Epoch 7/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9038 - loss: 0.2504 - val_accuracy: 0.8945 - val_loss: 0.2514
Epoch 8/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9054 - loss: 0.2415 - val_accu

[I 2025-05-12 14:27:19,790] Trial 0 finished with value: 0.9359145760536194 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 0 with value: 0.9359145760536194.


Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5323 - loss: 13.9264 - val_accuracy: 0.5532 - val_loss: 0.6877
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.5566 - loss: 0.6873 - val_accuracy: 0.5532 - val_loss: 0.6876
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5569 - loss: 0.6873 - val_accuracy: 0.5532 - val_loss: 0.6877
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5506 - loss: 0.6884 - val_accuracy: 0.5532 - val_loss: 0.6901
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5502 - loss: 0.6895 - val_accuracy: 0.5532 - val_loss: 0.6887
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5486 - loss: 0.6893 - val_accuracy: 0.5532 - val_loss: 0.6875
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5549 - loss: 0.6875 - val_accuracy: 0.5532 - val_loss: 0.6878
Epoch 8/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5577 - loss: 0.6871 - val_accuracy: 0.5532 - 

[I 2025-05-12 14:27:47,640] Trial 1 finished with value: 0.5531820058822632 and parameters: {'learning_rate': 0.1, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 0 with value: 0.9359145760536194.


...

Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.7173 - loss: 0.5027 - val_accuracy: 0.7953 - val_loss: 0.3951
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8021 - loss: 0.3854 - val_accuracy: 0.8669 - val_loss: 0.3328
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8681 - loss: 0.3144 - val_accuracy: 0.9052 - val_loss: 0.2549
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8966 - loss: 0.2510 - val_accuracy: 0.9186 - val_loss: 0.2030
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9170 - loss: 0.2128 - val_accuracy: 0.9297 - val_loss: 0.1828
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9366 - loss: 0.1727 - val_accuracy: 0.9435 - val_loss: 0.1468
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9416 - loss: 0.1455 - val_accuracy: 0.9519 - val_loss: 0.1419
Epoch 8/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9460 - loss: 0.1415 - val_accuracy: 0.9546 - v

[I 2025-05-12 14:47:21,552] Trial 28 finished with value: 0.9741877913475037 and parameters: {'learning_rate': 0.001, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 27 with value: 0.979083240032196.


Epoch 1/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.6965 - loss: 0.5343 - val_accuracy: 0.7802 - val_loss: 0.4200
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7838 - loss: 0.4236 - val_accuracy: 0.7944 - val_loss: 0.3961
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7953 - loss: 0.3981 - val_accuracy: 0.8282 - val_loss: 0.3717
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8305 - loss: 0.3681 - val_accuracy: 0.8518 - val_loss: 0.3433
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8496 - loss: 0.3438 - val_accuracy: 0.8678 - val_loss: 0.3240
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8662 - loss: 0.3135 - val_accuracy: 0.8736 - val_loss: 0.3007
Epoch 7/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8672 - loss: 0.3072 - val_accuracy: 0.8754 - val_loss: 0.2908
Epoch 8/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8799 - loss: 0.2832 - val_accu

[I 2025-05-12 14:48:09,761] Trial 29 finished with value: 0.9238985180854797 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 27 with value: 0.979083240032196.


Best trial:
  Accuracy: 0.979083240032196
  Hyperparameters:
    learning_rate: 0.001
    dropout_rate: 0.3
    batch_size: 32


In [ ]:
# Simpan best hyperparameters ke dalam DataFrame
best_params_df = pd.DataFrame([trial.params])
best_params_df['best_accuracy'] = trial.value  # tambahkan kolom akurasi terbaik

# Simpan ke file CSV
best_params_df.to_csv('best_hyperparameters.csv', index=False)


### CNN-LSTM Auto Hyperparameter Tuning

In [ ]:
# Load data dari file .npz
data_train_cnn_lstm = np.load('CNN_LSTM_train_data.npz')

X = data_train_cnn_lstm['data']
y = data_train_cnn_lstm['label']

X_half, _, y_half, _ = train_test_split(X, y, test_size=0.6, random_state=42)

# Bagi lagi menjadi training dan validation set (80% : 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_half, y_half, test_size=0.2, random_state=42
)

# Verifikasi ukuran
print(f"Total sample used: {len(X)}")
print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")


Total sample used: 22468
Train size: 7189, Validation size: 1798


In [ ]:
# Lihat distribusi kelas pada data training
classes_train, counts_train = np.unique(y_train, return_counts=True)
print("Distribusi kelas pada data TRAIN:")
for cls, count in zip(classes_train, counts_train):
    print(f"  Kelas {cls}: {count} sampel")

# Lihat distribusi kelas pada data validasi
classes_val, counts_val = np.unique(y_val, return_counts=True)
print("\nDistribusi kelas pada data VALIDASI:")
for cls, count in zip(classes_val, counts_val):
    print(f"  Kelas {cls}: {count} sampel")


Distribusi kelas pada data TRAIN:
  Kelas 0: 3218 sampel
  Kelas 1: 3971 sampel

Distribusi kelas pada data VALIDASI:
  Kelas 0: 779 sampel
  Kelas 1: 1019 sampel


In [ ]:

# === Objective Function ===
def objective(trial):
    # Hyperparameters to tune
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    dropout_rate = trial.suggest_categorical("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Define model
    model = Sequential([
        TimeDistributed(Conv2D(8, (3, 3), activation='relu', padding='same'),
                        input_shape=(32, 128, 4, 1), name="conv2d_1"),
        TimeDistributed(BatchNormalization(), name="batchnorm_1"),
        TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_1"),

        TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same'), name="conv2d_2"),
        TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_2"),

        TimeDistributed(Flatten(), name="flatten"),

        LSTM(64, return_sequences=True, name="lstm_1"),
        LSTM(32, return_sequences=False, name="lstm_2"),

        Dense(32, activation='relu', name="dense_1"),
        Dropout(dropout_rate, name="dropout_1"),

        Dense(1, activation='sigmoid', name="output")
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    # Return best validation accuracy as objective
    val_acc = max(history.history['val_accuracy'])
    return val_acc



In [ ]:
# === Run Optuna Study ===
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30)

# === Output best result ===
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-05-13 13:05:54,515] A new study created in memory with name: no-name-bafc68d2-75e8-43c6-ac7e-2e6c07b3e5d0
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 33s 111ms/step - accuracy: 0.5844 - loss: 0.6687 - val_accuracy: 0.8237 - val_loss: 0.4595
Epoch 2/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 12s 76ms/step - accuracy: 0.8229 - loss: 0.4425 - val_accuracy: 0.8704 - val_loss: 0.3328
Epoch 3/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.8827 - loss: 0.3151 - val_accuracy: 0.8704 - val_loss: 0.3209
Epoch 4/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.8932 - loss: 0.2790 - val_accuracy: 0.8215 - val_loss: 0.3937
Epoch 5/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.8973 - loss: 0.2623 - val_accuracy: 0.8398 - val_loss: 0.3716
Epoch 6/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.9072 - loss: 0.2400 - val_accuracy: 0.8543 - val_loss: 0.3463
Epoch 7/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.9245 - loss: 0.2001 - val_accuracy: 0.8804 - val_loss: 0.3251
Epoch 8/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.9331 - loss: 0.1790 - v

[I 2025-05-13 13:09:37,817] Trial 0 finished with value: 0.9532814025878906 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 0 with value: 0.9532814025878906.


Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 24s 172ms/step - accuracy: 0.5529 - loss: 0.7348 - val_accuracy: 0.5667 - val_loss: 0.6871
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.5541 - loss: 0.6879 - val_accuracy: 0.5667 - val_loss: 0.6843
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.5479 - loss: 0.6895 - val_accuracy: 0.5667 - val_loss: 0.6842
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - accuracy: 0.5503 - loss: 0.6889 - val_accuracy: 0.5667 - val_loss: 0.6844
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.5503 - loss: 0.6895 - val_accuracy: 0.5667 - val_loss: 0.6858
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 121ms/step - accuracy: 0.5576 - loss: 0.6866 - val_accuracy: 0.5667 - val_loss: 0.6916
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.5505 - loss: 0.6891 - val_accuracy: 0.5667 - val_loss: 0.6860
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.5579 - loss: 0.6872 - val_accura

[I 2025-05-13 13:11:11,742] Trial 1 finished with value: 0.5667408108711243 and parameters: {'learning_rate': 0.1, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 0 with value: 0.9532814025878906.


...

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 27s 172ms/step - accuracy: 0.7095 - loss: 0.5570 - val_accuracy: 0.8754 - val_loss: 0.3125
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - accuracy: 0.8779 - loss: 0.3001 - val_accuracy: 0.8771 - val_loss: 0.3097
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.8897 - loss: 0.2678 - val_accuracy: 0.8693 - val_loss: 0.3183
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 127ms/step - accuracy: 0.9059 - loss: 0.2331 - val_accuracy: 0.8782 - val_loss: 0.3020
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - accuracy: 0.9232 - loss: 0.2122 - val_accuracy: 0.9194 - val_loss: 0.2226
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 123ms/step - accuracy: 0.9329 - loss: 0.1780 - val_accuracy: 0.9260 - val_loss: 0.1752
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.9299 - loss: 0.1751 - val_accuracy: 0.7636 - val_loss: 0.4518
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - accuracy: 0.9429 - loss: 0.1383 - val_accuracy: 

[I 2025-05-13 15:18:29,485] Trial 28 finished with value: 0.9849833250045776 and parameters: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 128}. Best is trial 11 with value: 0.9911012053489685.


Epoch 1/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 26s 102ms/step - accuracy: 0.5697 - loss: 0.6772 - val_accuracy: 0.7903 - val_loss: 0.5224
Epoch 2/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.8066 - loss: 0.4815 - val_accuracy: 0.8810 - val_loss: 0.3260
Epoch 3/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.8827 - loss: 0.3300 - val_accuracy: 0.8643 - val_loss: 0.3268
Epoch 4/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.9130 - loss: 0.2622 - val_accuracy: 0.9088 - val_loss: 0.2490
Epoch 5/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - accuracy: 0.9176 - loss: 0.2357 - val_accuracy: 0.9199 - val_loss: 0.2207
Epoch 6/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.9150 - loss: 0.2216 - val_accuracy: 0.8932 - val_loss: 0.2721
Epoch 7/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.9194 - loss: 0.2171 - val_accuracy: 0.9310 - val_loss: 0.1863
Epoch 8/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.9312 - loss: 0.1979 - val

[I 2025-05-13 15:22:00,859] Trial 29 finished with value: 0.9532814025878906 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 11 with value: 0.9911012053489685.


Best trial:
  Accuracy: 0.9911012053489685
  Hyperparameters:
    learning_rate: 0.001
    dropout_rate: 0.1
    batch_size: 32


In [ ]:
# Simpan best hyperparameters ke dalam DataFrame
best_params_df = pd.DataFrame([trial.params])
best_params_df['best_accuracy'] = trial.value  # tambahkan kolom akurasi terbaik

# Simpan ke file CSV
best_params_df.to_csv('best_hyperparameters_cnn_lstm.csv', index=False)
